In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
import ipywidgets as widgets
from IPython.display import display, clear_output

In [ ]:
df = pd.read_csv('Dataset .csv')
df.info(), df.head()

In [ ]:
df['Cuisines'] = df['Cuisines'].fillna('Other')

In [ ]:
df['Cuisines'] = df['Cuisines'].apply(lambda x: x.split(',')[0].strip())
features = df[['Cuisines', 'Price range', 'Aggregate rating', 'Votes', 'Average Cost for two', 'City']].copy()
encoder = OneHotEncoder(sparse_output=False)
encoded_features = encoder.fit_transform(features[['Cuisines', 'Price range', 'City']])
encoded_features = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(['Cuisines', 'Price range', 'City']))

In [ ]:
scaler = MinMaxScaler()
rating_scaled = scaler.fit_transform(features[['Aggregate rating']])

In [ ]:
combined_features = np.hstack([encoded_features, rating_scaled])
restaurant_names = df['Restaurant Name'].reset_index(drop=True)
combined_features.shape, restaurant_names.head()

In [ ]:
price_bins = {
    1: (0, 200),
    2: (200, 500),
    3: (500, 1000),
    4: (1000, float('inf'))
}
def recommend_restaurants(cuisine_preference, budget_min, budget_max, city_preference, top_n=5):

    filtered_df = df[
        (df['Average Cost for two'] >= budget_min) & 
        (df['Average Cost for two'] <= budget_max) &
        (df['City'] == city_preference)
    ]

    if filtered_df.empty:
        return pd.DataFrame({'Message': ['No restaurants match your preferences.']})

    filtered_encoded = encoder.transform(filtered_df[['Cuisines', 'Price range', 'City']])
    filtered_ratings = scaler.transform(filtered_df[['Aggregate rating']])
    filtered_features = np.hstack([filtered_encoded, filtered_ratings])

    user_encoded = encoder.transform([[cuisine_preference, 2, city_preference]]) 
    avg_rating = np.array([[0.5]])
    user_profile = np.hstack([user_encoded, avg_rating])

    similarity_scores = cosine_similarity(user_profile, filtered_features)[0]
    top_indices = similarity_scores.argsort()[-top_n:][::-1]
    
    result_df = filtered_df.iloc[top_indices].copy()
    
    result_df['Similarity Score'] = similarity_scores[top_indices]
    
    return result_df[['Restaurant Name', 'Cuisines', 'Average Cost for two', 'Aggregate rating', 'Similarity Score']]



In [ ]:
cuisine_widget = widgets.Dropdown(
    options=sorted(df['Cuisines'].dropna().unique()),
    description='Cuisine:',
    style={'description_width': 'initial'}
)

city_widget = widgets.Dropdown(
    options=sorted(df['City'].dropna().unique()),
    description='City:',
    style={'description_width': 'initial'}
)

price_slider = widgets.IntRangeSlider(
    value=[100, 500],
    min=0,
    max=2000,
    step=50,
    description='Price Range (₹):',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='70%')
)

submit_button = widgets.Button(
    description='Get Recommendations',
    button_style='success',
    tooltip='Click to get recommendations'
)

output = widgets.Output()


In [ ]:
def on_submit(b):
    with output:
        clear_output()
        cuisine = cuisine_widget.value
        city = city_widget.value
        min_price, max_price = price_slider.value

        print(f"Showing recommendations for {cuisine} cuisine in {city} (₹{min_price}–₹{max_price})")
        display(recommend_restaurants(cuisine, min_price, max_price, city))
submit_button.on_click(on_submit)

In [37]:
display(widgets.VBox([
    cuisine_widget,
    city_widget,
    price_slider,
    submit_button,
    output
]))
